In [43]:
import json
import re
import random
from datetime import datetime
from collections import defaultdict
from transformers import pipeline
import pandas as pd

In [44]:
MAX_COMMENTI = 40000
MAX_TOKENS = 512
MEDIA_TOKENS_PER_WORD = 1.4
MAX_PAROLE = int(MAX_TOKENS / MEDIA_TOKENS_PER_WORD)

soggetti = {
    "stefania": ["stefania", "stefania cappa"],
    "paola": ["paola", "paola cappa"],
    "andrea": ["andrea", "andrea sempio", "sempio"],
    "alberto": ["alberto", "alberto stasi", "stasi"],
    "marco": ["marco", "marco poggi"],
}

In [45]:
def pulisci_testo(testo):
    testo = re.sub(r"http\S+", "", testo)  
    testo = re.sub(r"@\w+", "", testo)     
    testo = re.sub(r"[^a-zA-Zàèéìòùç0-9\s']", " ", testo)  
    testo = re.sub(r"\s+", " ", testo)    
    return testo.strip().lower()

def map_stars_to_sentiment(label):
    score = int(label[0])
    if score <= 2:
        return "neg"
    elif score == 3:
        return "neu"
    else:
        return "pos"

In [46]:
with open("comments.json", "r", encoding="utf-8") as f:
    all_comments = json.load(f)

In [47]:
valid_comments = [c for c in all_comments if "date" in c and c["date"] and c["content"].strip()]

for c in valid_comments:
    try:
        c["parsed_date"] = datetime.fromisoformat(c["date"].replace("Z", ""))
    except Exception:
        c["parsed_date"] = None

valid_comments = [c for c in valid_comments if c["parsed_date"]]

In [48]:
valid_comments.sort(key=lambda c: c["parsed_date"])
step = max(1, len(valid_comments) // MAX_COMMENTI)
sampled_comments = valid_comments[::step][:MAX_COMMENTI]
print(f"Commenti selezionati: {len(sampled_comments)} (ogni {step} commenti)")

Commenti selezionati: 40000 (ogni 2 commenti)


In [49]:
sentiment_model = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

In [50]:
sentiment_per_soggetto = defaultdict(lambda: {"pos": 0, "neg": 0, "neu": 0})
error_count = 0
match_count = 0

for i, c in enumerate(sampled_comments):
    testo = pulisci_testo(c["content"])
    parole = testo.split()
    if len(parole) > MAX_PAROLE:
        testo = " ".join(parole[:MAX_PAROLE])

    for soggetto, varianti in soggetti.items():
        if any(v in testo for v in varianti):
            try:
                result = sentiment_model(testo)[0]
                label = map_stars_to_sentiment(result["label"])
                sentiment_per_soggetto[soggetto][label] += 1
                match_count += 1
            except Exception as e:
                error_count += 1
                print(f"⚠️ Errore al commento #{i}: {testo[:80]}... → {e}")
            break 

print(f"Commenti associati a soggetti: {match_count}")
print(f"Errori durante l'analisi: {error_count}")

Token indices sequence length is longer than the specified maximum sequence length for this model (542 > 512). Running this sequence through the model will result in indexing errors


⚠️ Errore al commento #1439: se il coinvolgimento di sempio fosse confermato questo caso sarebbe molto più si... → The size of tensor a (542) must match the size of tensor b (512) at non-singleton dimension 1
⚠️ Errore al commento #2392: la butto lì chiara poggi ha lasciato la colazione a metà escludendo che l'assass... → The size of tensor a (518) must match the size of tensor b (512) at non-singleton dimension 1
⚠️ Errore al commento #3894: ma non è stata la difesa di stasi a richiedere la riapertura delle indagini anch... → The size of tensor a (517) must match the size of tensor b (512) at non-singleton dimension 1
⚠️ Errore al commento #13693: stefania non cercava di ottenere da chiara proprio un incontro con il figlio del... → The size of tensor a (534) must match the size of tensor b (512) at non-singleton dimension 1
⚠️ Errore al commento #15939: invito a riflettere su tre aspetti fondamentali dell'attuale situazione investig... → The size of tensor a (513) must match the size 

In [51]:
df = pd.DataFrame(sentiment_per_soggetto).T
df["totale"] = df.sum(axis=1)
df["% negativo"] = 100 * df["neg"] / df["totale"]

In [52]:
print(df.sort_values("% negativo", ascending=False))

          pos   neg  neu  totale  % negativo
alberto   677  2619  615    3911   66.964971
andrea    447  1903  891    3241   58.716446
paola      31   103   51     185   55.675676
stefania  123   251  171     545   46.055046
marco      67   143  108     318   44.968553
